In [15]:
from libsvm.svmutil import *
from liblinear.liblinearutil import *
from sklearn import linear_model
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import pandas as pd
import itertools
import random
import sys

In [25]:
df_tr = pd.read_csv("ML_FP_train.csv")
df_tr = df_tr.drop(['Album_type', 'Licensed', 'official_video', 'Track', 'Album', \
    'Uri', 'Url_spotify', 'Url_youtube', 'Comments', 'Description', 'Title', \
    'Channel', 'Composer', 'Artist', 'id', 'Views', 'Likes', 'Stream'], axis=1)
# df_tr.shape (17170, 29)

random_rows = df_tr.sample(n=10)
print(random_rows)

df_tr = df_tr.fillna(df_tr.median()) #處理缺失值

       Danceability    Energy   Key  Loudness  Speechiness  Acousticness  \
5214            0.0  0.002924   2.0   -12.218       0.0330  1.031617e-01   
4200            9.0       NaN   9.0    -3.565       0.1300  1.025032e-01   
6100            5.0  0.370146   7.0       NaN       0.0594  3.089158e-07   
1088            7.0  0.078954  10.0   -11.307       0.0641  1.488694e-08   
15096           2.0  0.487443   8.0    -4.768       0.0536  1.092727e-03   
16865           8.0  0.605496   1.0    -4.495       0.2070  1.264934e-02   
4378            2.0  0.057961   7.0       NaN       0.0377  6.837978e-01   
16956           7.0  0.207475   0.0    -5.515       0.0438  2.500470e-01   
4715            7.0  0.080622   7.0    -6.891       0.0550  1.488694e-02   
14213           9.0  0.294080  10.0    -6.082       0.1450  4.410944e-03   

       Instrumentalness  Liveness  Valence    Tempo  Duration_ms  
5214                NaN  0.001030    0.179  110.823     315973.0  
4200           0.000000      

In [26]:
df_tr.describe()

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms
count,17170.000000,1.717000e+04,17170.000000,17170.000000,17170.000000,1.717000e+04,17170.000000,17170.000000,17170.000000,17170.000000,1.717000e+04
mean,4.586022,3.298228e-01,5.196506,-7.398377,0.089008,9.865013e-02,0.046612,0.029045,0.526615,120.797212,2.211933e+05
std,2.858980,2.243601e-01,3.185355,4.205446,0.096486,2.078188e-01,0.177396,0.106644,0.225816,27.364553,1.116707e+05
min,0.000000,8.365427e-15,0.000000,-46.251000,0.000000,1.367631e-18,0.000000,0.000003,0.000000,0.000000,3.098500e+04
25%,2.000000,1.601030e-01,2.000000,-8.124000,0.038000,2.409325e-04,0.000000,0.000982,0.368000,100.071000,1.851942e+05
50%,5.000000,2.954083e-01,5.000000,-6.472000,0.051400,6.967871e-03,0.000002,0.001907,0.531000,119.995500,2.120300e+05
75%,7.000000,4.600996e-01,8.000000,-5.175000,0.089400,6.643013e-02,0.000135,0.008000,0.689000,135.065250,2.403968e+05
max,9.000000,1.000000e+00,10.000000,0.829000,0.964000,9.880479e-01,1.000000,1.000000,0.993000,243.372000,4.581483e+06


In [27]:
dep_var = 'Danceability'
indep_var = ['Energy', 'Key', 'Loudness', 'Speechiness',
       'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Duration_ms']

y = df_tr[dep_var]
x = df_tr[indep_var]

# with statsmodels
x = sm.add_constant(x) # adding a constant
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:           Danceability   R-squared:                       0.281
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                     669.8
Date:                Tue, 16 May 2023   Prob (F-statistic):               0.00
Time:                        22:54:22   Log-Likelihood:                -39570.
No. Observations:               17170   AIC:                         7.916e+04
Df Residuals:                   17159   BIC:                         7.925e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                5.6737      0.125  